## Libraries

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from llama_index.core import SimpleDirectoryReader
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

## Creating chunks

In [2]:
text_splitter = CharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

dynamic_path = f'./data/'
loader = DirectoryLoader(dynamic_path, glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
splits = text_splitter.split_documents(documents=documents)


Created a chunk of size 1005, which is longer than the specified 1000
Created a chunk of size 1442, which is longer than the specified 1000
Created a chunk of size 1325, which is longer than the specified 1000
Created a chunk of size 1860, which is longer than the specified 1000
Created a chunk of size 1364, which is longer than the specified 1000
Created a chunk of size 1245, which is longer than the specified 1000
Created a chunk of size 2502, which is longer than the specified 1000
Created a chunk of size 2590, which is longer than the specified 1000
Created a chunk of size 1252, which is longer than the specified 1000
Created a chunk of size 1307, which is longer than the specified 1000
Created a chunk of size 2559, which is longer than the specified 1000
Created a chunk of size 4018, which is longer than the specified 1000
Created a chunk of size 4425, which is longer than the specified 1000
Created a chunk of size 1508, which is longer than the specified 1000
Created a chunk of s

## Creating Embeddings if not created

In [3]:
embedding = OpenAIEmbeddings()

persist_directory = 'docs/chroma/'


vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
) 

# save the database so we can use it later
# vectordb.persist()

# check that the database have been created and get the number of documents
print(vectordb._collection.count())

737


## Similarity search with score 

In [4]:
# similarity search
question = "how can i setup auto label printing"


docs = vectordb.similarity_search_with_score(question,k=3)

for result in docs:
    print("\n")
    print(result[1])
    print(result[0].page_content)



0.2765898108482361
5. **Install QZ Tray**:
   - Run the installation file downloaded in the previous step.
   - Ensure that Java SE Development Kit is installed on your machine.

6. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

7. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that "Automatically Start" option is enabled.

### Step 4: Label Printing

8. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.

---

These detailed instructions cover the setup process for auto label printing using a Zebra printer. Following these steps ensures smooth and efficient label printing operations.

For further assistance, refer to the [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).


0.2765898108482361
5. **Install QZ Tray**:
   - Run the installation file down

## Loading existing vectorDB (If embeddings are already created) and querying

In [5]:
persist_directory = 'docs/chroma/'
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)
# from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

result = qa_chain.invoke({"query": question})
print(result["result"])

To set up auto label printing with a Zebra printer, follow these steps:

1. **Hardware Setup**:
   - Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install the Zebra Printer Driver or a ZPL supported printer driver on your machine.

3. **Install QZ Tray**:
   - Run the installation file downloaded in the previous step.
   - Ensure that Java SE Development Kit is installed on your machine.

4. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

5. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

6. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.

For more detailed instructions, you can refer to the [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).


## Using templates to form a chain and get specific answer

In [6]:
template = """Address the user as vinner and Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

print(result["result"])

To set up auto label printing with a Zebra printer, follow these steps:

1. **Hardware Setup**:
   - Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install the Zebra Printer Driver or a ZPL supported printer driver on your machine.

3. **Install QZ Tray**:
   - Run the installation file downloaded in the previous step.
   - Ensure that Java SE Development Kit is installed on your machine.

4. **Start QZ Tray**:
   - After successful installation, navigate to Start > All Programs > QZ Tray.

5. **Automatic Startup**:
   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

6. **Test Print**:
   - Once all setup steps are completed, attempt to print the label from the panel.

For more detailed instructions, you can refer to the [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).


## Using memory to store the conversation and get the answer

In [14]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb.as_retriever(),
    memory=memory
)
question = "how can i setup auto label printing"
result = qa.invoke({"question": question})

print(result['answer'])

To set up auto label printing with a Zebra printer, follow these steps:

1. **Hardware Setup**:
   - Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install QZ Tray by running the installation file downloaded in the previous step. Make sure Java SE Development Kit is installed on your machine.
   - Start QZ Tray by navigating to Start > All Programs > QZ Tray after successful installation.
   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

3. **Label Printing**:
   - Once all setup steps are completed, attempt to print the label from the panel.

For more detailed information, you can refer to the [Zebra Printer Settings for Auto Printing guide](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).


## ChatHistory

In [15]:
question = "Can you provide me link for the same"
result = qa.invoke({"question": question})
# print(result['answer'])
print(result)



{'question': 'Can you provide me link for the same', 'chat_history': [HumanMessage(content='how can i setup auto label printing'), AIMessage(content='To set up auto label printing with a Zebra printer, follow these steps:\n\n1. **Hardware Setup**:\n   - Connect the Zebra printer to your machine using a USB cable.\n\n2. **Software Setup**:\n   - Install QZ Tray by running the installation file downloaded in the previous step. Make sure Java SE Development Kit is installed on your machine.\n   - Start QZ Tray by navigating to Start > All Programs > QZ Tray after successful installation.\n   - Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.\n\n3. **Label Printing**:\n   - Once all setup steps are completed, attempt to print the label from the panel.\n\nFor more detailed information, you can refer to the [Zebra Printer Settings for Auto Printing guide](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-set

## Ignore from here

In [9]:
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.schema.document import Document
from typing import List
class MyVectorStoreRetriever(VectorStoreRetriever):
    # See https://github.com/langchain-ai/langchain/blob/61dd92f8215daef3d9cf1734b0d1f8c70c1571c3/libs/langchain/langchain/vectorstores/base.py#L500
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        docs_and_similarities = (
            self.vectorstore.similarity_search_with_relevance_scores(
                query, **self.search_kwargs
            )
        )

        # Make the score part of the document metadata
        for doc, similarity in docs_and_similarities:
            doc.metadata["score"] = similarity

        docs = [doc for doc, _ in docs_and_similarities]
        return docs

In [12]:
# retriever = MyVectorStoreRetriever(
#    vectorstore=vectordb,
#    search_type="similarity_score_threshold",
#    search_kwargs={"score_threshold": similarity_threshold, "k": 3},
# )
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    # retriever=vectordb.as_retriever(),
    retriever=MyVectorStoreRetriever(
        vectorstore=vectordb,
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.5, "k": 3},
    ),
    memory=memory
)
question = "how can i setup auto label printing"
result = qa.invoke({"question": question})

print(result['answer'])


To set up auto label printing with a Zebra printer, you can follow these steps:

1. **Hardware Setup**:
   - Plugin the Zebra Printer USB Cable: Connect the Zebra printer to your machine using a USB cable.

2. **Software Setup**:
   - Install QZ Tray: Run the installation file downloaded in the previous step. Ensure that Java SE Development Kit is installed on your machine.
   - Start QZ Tray: After successful installation, navigate to Start > All Programs > QZ Tray.
   - Automatic Startup: Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.

3. **Label Printing**:
   - Test Print: Once all setup steps are completed, attempt to print the label from the panel.

For more detailed instructions, you can refer to the [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdesk.com/support/solutions/articles/9000210122-zebra-printer-settings-for-auto-printing).


In [13]:
question="what was the last question i asked"
result = qa.invoke({"question": question})
print(result)

{'question': 'what was the last question i asked', 'chat_history': [HumanMessage(content='how can i setup auto label printing'), AIMessage(content='To set up auto label printing with a Zebra printer, you can follow these steps:\n\n1. **Hardware Setup**:\n   - Plugin the Zebra Printer USB Cable: Connect the Zebra printer to your machine using a USB cable.\n\n2. **Software Setup**:\n   - Install QZ Tray: Run the installation file downloaded in the previous step. Ensure that Java SE Development Kit is installed on your machine.\n   - Start QZ Tray: After successful installation, navigate to Start > All Programs > QZ Tray.\n   - Automatic Startup: Right-click on the QZ Tray icon and ensure that the "Automatically Start" option is enabled.\n\n3. **Label Printing**:\n   - Test Print: Once all setup steps are completed, attempt to print the label from the panel.\n\nFor more detailed instructions, you can refer to the [Zebra Printer Settings for Auto Printing](https://vinculumhelpdesk.freshdes